In [1]:
from modules.data_visualizer import ImageVisualizer
from modules.data_loader import DataLoaderCreator
from modules.data_prediction import ComprehensiveImagePredictor
from modules.model_eval import ModelEvaluator
from modules.model_performance import ModelPerformanceVisualizer
from modules.model_trainer import ModelTrainer

### Initialize Dataset

In [2]:
SPLITTED_DATASET = "DATASET"
train_dir = "DATASET/train"
test_dir  = "DATASET/test"

train_dir,test_dir

visualizer = ImageVisualizer(SPLITTED_DATASET)
visualizer.visualize_random_image()

('DATASET/train', 'DATASET/test')

In [3]:
import torch
from torchvision import models
from torchvision import transforms
from torchvision.transforms import InterpolationMode

weights = models.MobileNet_V2_Weights.DEFAULT

print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
    print(f"Device count: {torch.cuda.device_count()}")
else:
    print("CUDA is not available. Using CPU.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ASL_mobilenet = models.mobilenet_v2(weights=weights).to(device)



loader_creator = DataLoaderCreator(train_dir=train_dir,
                                   test_dir=test_dir,
                                   transform=train_transforms)

train_dataloader, test_dataloader, class_names = loader_creator.create_dataloaders(batch_size=32)

class_names_count = len(loader_creator.get_class_names())

print(f"Class names: {loader_creator.get_class_names()}")
print(f"Class names Count: {class_names_count}")




CUDA available: True
CUDA version: 12.1
Device name: NVIDIA GeForce GTX 1070
Device count: 1
Class names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
Class names Count: 35


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((232, 232), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((232, 232), interpolation=InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

visualizer.plot_transformed_images(train_transforms,n=3, seed=None)


#### Train 

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

for param in ASL_mobilenet.features.parameters():
    param.requires_grad = False

loader_creator = DataLoaderCreator(train_dir=train_dir,
                                   test_dir=test_dir,
                                   transform=train_transforms)

train_dataloader, test_dataloader, class_names = loader_creator.create_dataloaders(batch_size=32)

class_names_count = len(loader_creator.get_class_names())

print(f"Class names: {loader_creator.get_class_names()}")
print(f"Class names Count: {class_names_count}")

trainer = ModelTrainer()

NUM_EPOCHS = 2
EARLY_STOPPING = 1
LEARNING_RATE = 0.001

ASL_mobilenet.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=class_names_count,
                    bias=True)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=ASL_mobilenet.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

TRAINED_MODEL, RESULTS = trainer.train(
    model=ASL_mobilenet,
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=NUM_EPOCHS,
    scheduler=scheduler,
    patience=EARLY_STOPPING 
)

visualizer = ModelPerformanceVisualizer(RESULTS)
y_true, y_pred = visualizer.get_preds(model=TRAINED_MODEL, dataloader=test_dataloader, device=device)
visualizer.plot_all(y_true=y_true, y_pred=y_pred, classes=class_names, save_path="model_performance/plot.jpg")

#### Load the Model

In [4]:
import torch
from torchvision.models import mobilenet_v2

num_classes = 35
model = mobilenet_v2(weights=None)
model.classifier[1] = torch.nn.Linear(model.last_channel, num_classes)
state_dict = torch.load("models/nums_alphs_35_mobilenetv2.pth")
model.load_state_dict(state_dict)

<All keys matched successfully>

#### Multilple Image Prediciton in test dir

In [ ]:
predictor = ComprehensiveImagePredictor(model, class_names, train_transforms)
predictor.predict_and_visualize_multiple(model=model, directory=test_dir, transformer=train_transforms, num_samples=9, seed=None)

file_path = r"D:\SignLanguage\NOTEBOOKS\DATASET\test\2\C_2 (44).jpg"
# predictor.predict_and_plot_image(file_path)

#### Model Eval

In [9]:
evaluator = ModelEvaluator(model)
results = evaluator.evaluate_and_report(test_dataloader, class_names)

print(f"Accuracy: {results['accuracy']:.4f}")
print("Classification Report:")
print(results['classification_report'])

Accuracy: 0.9819
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        60
           2       0.98      0.97      0.97        60
           3       1.00      1.00      1.00        60
           4       0.93      0.93      0.93        60
           5       0.93      0.93      0.93        60
           6       0.89      0.98      0.94        60
           7       0.98      0.98      0.98        60
           8       0.97      1.00      0.98        60
           9       1.00      0.98      0.99        60
           A       1.00      1.00      1.00        60
           B       1.00      1.00      1.00        60
           C       1.00      1.00      1.00        60
           D       0.97      1.00      0.98        60
           E       1.00      1.00      1.00        60
           F       0.97      1.00      0.98        60
           G       1.00      1.00      1.00        60
           H       1.00      1.00      1.

#### Prep model for Android

#### Save for android

In [ ]:
import torch
import torchvision.models as models

OUT_FEATURES = 35
MODEL_PATH = "models/nums_alphs_35_mobilenetv2.pth"
MODEL_NAME = "mnetV2_35"

model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=OUT_FEATURES)

model.load_state_dict(torch.load(MODEL_PATH))

model.eval()

example_input = torch.rand(1, 3, 224, 224)  # Batch size 1, 3 channels, 224x224 pixels
traced_model = torch.jit.trace(model, example_input)

optimized_model = torch.utils.mobile_optimizer.optimize_for_mobile(traced_model)

optimized_model._save_for_lite_interpreter(f"models/{MODEL_NAME}.ptl")


In [47]:
import torch

lite_model = torch.jit.load(f"models/{MODEL_NAME}.ptl")

example_input = torch.rand(1, 3, 224, 224)  # Example input with the same size used for tracing
output = lite_model(example_input)

print(f"Output shape: {output.shape}")

Output shape: torch.Size([1, 35])
